In [5]:
"""
FASE 1: Problema Directo con Efectos de Surfactante
====================================================
Modelo simplificado 2D basado en Avilés-Rojas & Hurtado (2025)

Ecuaciones:
- Equilibrio: Div(F·S) = 0
- Tensor de esfuerzos: S = S_el + (P_γ - P_alv)·J·C⁻¹
- Presión de colapso: P_γ = (2γ/R_I)·(Φ_R/Φ)^(1/3)

Simplificaciones:
- 2D (estado plano de deformación)
- Quasi-estático (sin dinámica de Darcy)
- Surfactante en régimen insoluble (γ función de Γ)
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile
import matplotlib.pyplot as plt

fd.parameters["form_compiler"]["quadrature_degree"] = 6

# ============================================================
# 1. PARÁMETROS DEL MODELO (Tabla 1 del paper)
# ============================================================

# Geometría alveolar
Phi_R = 0.74          # Porosidad de referencia [-]
R_RI = 0.11           # Radio interno de referencia [mm]

# Material Fung (tejido)
c_fung = 2.5       # [kPa] (paper usa 2.5 kPa, escalamos para 2D)
a_fung = 0.433        # [-]
b_fung = -0.61        # [-]

# Surfactante (Otis et al.)
gamma_0 = 70.0e-6     # Tensión superficial sin surfactante [N/mm = kPa·mm]
gamma_inf = 22.2e-6   # γ en umbral Langmuir/insoluble [N/mm]
gamma_min = 0.0       # Tensión mínima [N/mm]
Gamma_inf = 3.0e-9    # Concentración umbral [g/mm²]
m1 = 47.8e-6          # Pendiente régimen Langmuir [N/mm]
m2 = 140.0e-6         # Pendiente régimen insoluble [N/mm]

# Concentración máxima dinámica (Ec. 51)
Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

print("="*60)
print("PARÁMETROS DEL MODELO")
print("="*60)
print(f"Φ_R = {Phi_R}, R_I = {R_RI} mm")
print(f"Fung: c={c_fung} kPa, a={a_fung}, b={b_fung}")
print(f"Surfactante: γ₀={gamma_0*1e6:.1f}, γ_inf={gamma_inf*1e6:.1f} μN/mm")
print(f"Γ_inf={Gamma_inf:.2e}, Γ_max={Gamma_max:.2e} g/mm²")

# ============================================================
# 2. MALLA Y ESPACIOS DE FUNCIONES
# ============================================================

# Dominio: lámina rectangular (representa parénquima)
Lx, Ly = 10.0, 10.0  # [mm]
nx, ny = 20, 20

mesh = fd.RectangleMesh(nx, ny, Lx, Ly)
print(f"\nMalla: {nx}x{ny} elementos, dominio {Lx}x{Ly} mm")

V = fd.VectorFunctionSpace(mesh, "CG", 2) # Desplazamiento
Q = fd.FunctionSpace(mesh, "DG", 0)       # Concentración surfactante (por elemento)

u = fd.Function(V, name="Displacement")
v = fd.TestFunction(V)
Gamma = fd.Function(Q, name="Surfactant_Concentration")

coords_initial = mesh.coordinates.dat.data_ro.copy()

dim = mesh.geometric_dimension()
I = fd.Identity(dim)

# ============================================================
# 3. MODELO CONSTITUTIVO
# ============================================================

# Cinemática
F = I + fd.grad(u)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)
# Tensor de Green-Lagrange
E = fd.variable(0.5 * (C - I))

# Invariantes de E
J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

# Energía elástica Fung (Ec. 44)
Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Porosidad Lagrangiana: Φ = J - 1 + Φ_R (Ec. 17)
Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)

# Tensión superficial según modelo de Otis (Ec. 48)
ratio = Gamma / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma = fd.conditional(fd.lt(Gamma, Gamma_inf), gamma_lang, gamma_insol)
gamma = fd.max_value(gamma, gamma_min)

# Presión de colapso: P_γ = (2γ/R_I)·(Φ_R/Φ)^(1/3) (Ec. 43)
P_gamma = (2.0 * gamma / R_RI) * ((Phi_R / Phi)**(1.0/3.0))

P_alv = fd.Constant(0.0)

# Tensor de esfuerzos total (Ec. 42) S = S_el + (P_γ - P_alv)·J·C⁻¹
S_total = S_el + (P_gamma - P_alv) * J * invC

# ============================================================
# 4. FORMULACIÓN VARIACIONAL
# ============================================================

# Residuo mecánico (Ec. 70)
F_stress = F * S_total
Residual = fd.inner(F_stress, fd.grad(v)) * fd.dx

bcs = [
    fd.DirichletBC(V, fd.Constant((0.0, 0.0)), 1),
    fd.DirichletBC(V.sub(1), 0.0, 3),
]

problem = fd.NonlinearVariationalProblem(Residual, u, bcs=bcs)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 100,
    "snes_atol": 1e-8,
    "snes_rtol": 1e-6,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# ============================================================
# 5. SIMULACIÓN DE INFLACIÓN
# ============================================================

print("\n" + "="*60)
print("SIMULACIÓN DE INFLACIÓN")
print("="*60)

Gamma.assign(fd.Constant(Gamma_max))
u.assign(fd.Constant((0.0, 0.0)))

output_dir = "phase1_results"
os.makedirs(output_dir, exist_ok=True)

gamma_field = fd.Function(Q, name="Surface_Tension")
P_gamma_field = fd.Function(Q, name="Collapse_Pressure")
J_field = fd.Function(Q, name="Jacobian")

P_target = 1.0 # 1.0 kPa de carga máxima
n_steps = 40

steps_list = [0]
pressures = [0.0]
volumes = [0.0]
u_norms = [0.0]
energies = [0.0]

outfile = VTKFile(os.path.join(output_dir, "inflation.pvd"))

for step in range(1, n_steps + 1):
    P_val = P_target * step / n_steps
    P_alv.assign(P_val)
    
    print(f"\nStep {step}/{n_steps}: P_alv = {P_val:.4f} kPa")
    
    try:
        solver.solve()
    except fd.ConvergenceError:
        print("  ⚠ No convergió")
        break
    
    # Actualizar surfactante
    J_curr = fd.project(fd.det(I + fd.grad(u)), Q)
    Phi_curr = fd.project(fd.max_value(J_curr - 1.0 + Phi_R, 1e-4), Q)
    
    # Densidad de área interfacial: A = (3/R_I)·Φ_R^(1/3)·Φ^(2/3) (Ec. 39)
    # Ratio = (Phi_R / Phi)^(2/3)
    area_ratio = (Phi_R / Phi_curr)**(2.0/3.0)
    Gamma_new = fd.project(Gamma_max * area_ratio, Q)
    Gamma_new_data = np.clip(Gamma_new.dat.data[:], Gamma_inf, Gamma_max)
    Gamma.dat.data[:] = Gamma_new_data
    
    # Campos
    J_field.assign(J_curr)
    
    ratio_curr = Gamma / Gamma_inf
    gamma_expr = fd.conditional(fd.lt(Gamma, Gamma_inf), 
                                 gamma_0 - m1 * ratio_curr,
                                 gamma_inf - m2 * (ratio_curr - 1.0))
    gamma_expr = fd.max_value(gamma_expr, gamma_min)
    gamma_field.assign(fd.project(gamma_expr, Q))
    
    Phi_safe = fd.max_value(J_curr - 1.0 + Phi_R, 1e-4)
    P_gamma_expr = (2.0 * gamma_expr / R_RI) * ((Phi_R / Phi_safe)**(1.0/3.0))
    P_gamma_field.assign(fd.project(P_gamma_expr, Q))
    
    # Métricas
    V_change = fd.assemble((J_curr - 1.0) * fd.dx)
    u_L2 = fd.norm(u)
    energy = fd.assemble(Psi_el * fd.dx)
    
    steps_list.append(step)
    pressures.append(P_val)
    volumes.append(V_change)
    u_norms.append(u_L2)
    energies.append(energy)
    
    J_min, J_max = J_field.dat.data_ro.min(), J_field.dat.data_ro.max()
    gamma_avg = fd.assemble(gamma_field * fd.dx) / (Lx * Ly)
    
    print(f"  ||u||_L2 = {u_L2:.6e}, J ∈ [{J_min:.4f}, {J_max:.4f}]")
    print(f"  γ_avg = {gamma_avg*1e6:.2f} μN/mm, ΔV = {V_change:.4f} mm²")
    
    outfile.write(u, Gamma, gamma_field, P_gamma_field, J_field)

# ============================================================
# 6. GUARDAR MALLA DEFORMADA
# ============================================================

print("\n" + "="*60)
print("GUARDANDO RESULTADOS")
print("="*60)

V_coords = fd.VectorFunctionSpace(mesh, "CG", 1)
# Aquí saltará el warning de interpolate, es normal.
u_at_nodes = fd.interpolate(u, V_coords)
coords_deformed = coords_initial + u_at_nodes.dat.data_ro

mesh_deformed = fd.Mesh(mesh.coordinates.copy(deepcopy=True))
mesh_deformed.coordinates.dat.data[:] = coords_deformed

deformed_file = VTKFile(os.path.join(output_dir, "mesh_deformed.pvd"))
V_def = fd.VectorFunctionSpace(mesh_deformed, "CG", 1)
zero_field = fd.Function(V_def, name="zero")
deformed_file.write(zero_field)

np.savez(os.path.join(output_dir, "forward_data.npz"),
         u_data=u.dat.data_ro[:],
         Gamma_data=Gamma.dat.data_ro[:],
         P_alv_final=float(P_alv),
         coords_initial=coords_initial,
         coords_deformed=coords_deformed,
         steps=np.array(steps_list),
         pressures=np.array(pressures),
         volumes=np.array(volumes),
         u_norms=np.array(u_norms),
         energies=np.array(energies))

# ============================================================
# 7. GRÁFICAS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].plot(pressures, volumes, 'b-o', markersize=4)
axes[0, 0].set_xlabel('Presión [kPa]')
axes[0, 0].set_ylabel('ΔV [mm²]')
axes[0, 0].set_title('Curva P-V')
axes[0, 0].grid(True)

axes[0, 1].plot(steps_list, u_norms, 'r-o', markersize=4)
axes[0, 1].set_xlabel('Paso')
axes[0, 1].set_ylabel('||u||_L2 [mm]')
axes[0, 1].set_title('Norma L2 del Desplazamiento')
axes[0, 1].grid(True)

axes[1, 0].plot(steps_list[1:], energies[1:], 'g-o', markersize=4)
axes[1, 0].set_xlabel('Paso')
axes[1, 0].set_ylabel('Energía [kPa·mm²]')
axes[1, 0].set_title('Energía Elástica')
axes[1, 0].grid(True)

axes[1, 1].plot(coords_initial[:, 0], coords_initial[:, 1], 'b.', markersize=1, alpha=0.5, label='Inicial')
axes[1, 1].plot(coords_deformed[:, 0], coords_deformed[:, 1], 'r.', markersize=1, alpha=0.5, label='Deformado')
axes[1, 1].set_xlabel('x [mm]')
axes[1, 1].set_ylabel('y [mm]')
axes[1, 1].set_title('Configuración Inicial vs Deformada')
axes[1, 1].legend()
axes[1, 1].set_aspect('equal')
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "convergence_plots.png"), dpi=150)
plt.close()

# ============================================================
# 8. RESUMEN
# ============================================================

print(f"\nArchivos en: {output_dir}/")

displacement_magnitude = np.linalg.norm(coords_deformed - coords_initial, axis=1)

print("\n" + "="*60)
print("RESUMEN")
print("="*60)
print(f"  Pasos: {len(steps_list)-1}/{n_steps}")
print(f"  P_alv final: {pressures[-1]:.4f} kPa")
print(f"  ||u||_L2 final: {u_norms[-1]:.6e} mm")
print(f"  ΔV final: {volumes[-1]:.4f} mm²")
print(f"  Desplazamiento máx: {displacement_magnitude.max():.6e} mm")
print(f"  Desplazamiento prom: {displacement_magnitude.mean():.6e} mm")

print("\nFASE 1 COMPLETADA")

PARÁMETROS DEL MODELO
Φ_R = 0.74, R_I = 0.11 mm
Fung: c=2.5 kPa, a=0.433, b=-0.61
Surfactante: γ₀=70.0, γ_inf=22.2 μN/mm
Γ_inf=3.00e-09, Γ_max=3.48e-09 g/mm²

Malla: 20x20 elementos, dominio 10.0x10.0 mm

SIMULACIÓN DE INFLACIÓN

Step 1/40: P_alv = 0.0250 kPa
  ||u||_L2 = 6.850264e-01, J ∈ [1.0021, 1.0185]
  γ_avg = 2.31 μN/mm, ΔV = 1.6104 mm²

Step 2/40: P_alv = 0.0500 kPa
  ||u||_L2 = 1.355930e+00, J ∈ [1.0030, 1.0370]
  γ_avg = 4.52 μN/mm, ΔV = 3.2070 mm²

Step 3/40: P_alv = 0.0750 kPa
  ||u||_L2 = 2.014417e+00, J ∈ [1.0027, 1.0556]
  γ_avg = 6.64 μN/mm, ΔV = 4.7928 mm²

Step 4/40: P_alv = 0.1000 kPa
  ||u||_L2 = 2.660934e+00, J ∈ [1.0012, 1.0741]
  γ_avg = 8.68 μN/mm, ΔV = 6.3679 mm²

Step 5/40: P_alv = 0.1250 kPa
  ||u||_L2 = 3.295877e+00, J ∈ [0.9985, 1.0927]
  γ_avg = 10.63 μN/mm, ΔV = 7.9324 mm²

Step 6/40: P_alv = 0.1500 kPa
  ||u||_L2 = 3.919594e+00, J ∈ [0.9947, 1.1112]
  γ_avg = 12.51 μN/mm, ΔV = 9.4864 mm²

Step 7/40: P_alv = 0.1750 kPa
  ||u||_L2 = 4.532398e+00, J ∈ [0.98

/home/ebasu/venv-firedrake/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.



Archivos en: phase1_results/

RESUMEN
  Pasos: 40/40
  P_alv final: 1.0000 kPa
  ||u||_L2 final: 1.982987e+01 mm
  ΔV final: 55.1354 mm²
  Desplazamiento máx: 3.645214e+00 mm
  Desplazamiento prom: 1.760343e+00 mm

FASE 1 COMPLETADA


In [11]:
"""
FASE 2 (DEFINITIVA): Recuperación visualizable en ParaView
==========================================================
Estrategia: Inverse Motion sobre Geometría Deformada

Objetivo:
  Generar un archivo .pvd que muestre la GEOMETRÍA DEFORMADA
  y contenga un vector 'Recovery_Vector' para recuperar la forma original.

Uso en ParaView:
  1. Abrir 'inverse_visual_ready.pvd'.
  2. Verás la malla distorsionada (deformada).
  3. Aplica filtro 'Warp by Vector' seleccionando 'Recovery_Vector'.
  4. La malla volverá a su forma cuadrada original.
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile

# ============================================================
# 1. PARÁMETROS FÍSICOS
# ============================================================

# Geometría y Material
Phi_R = 0.74
R_RI = 0.11
c_fung = 2.5
a_fung = 0.433
b_fung = -0.61

# Surfactante
gamma_0 = 70.0e-6
gamma_inf = 22.2e-6
gamma_min = 0.0
Gamma_inf = 3.0e-9
m1 = 47.8e-6
m2 = 140.0e-6

print("="*60)
print("FASE 2: INVERSE MOTION SOBRE MALLA DEFORMADA")
print("="*60)

# ============================================================
# 2. CARGAR "IMAGEN" (DATOS DE LA MALLA DEFORMADA)
# ============================================================

input_dir = "phase1_results"
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"No se encuentra {input_dir}. Ejecuta phase1_forward.py primero.")

# Cargamos los datos numéricos de la Fase 1
print(f"Cargando datos desde {input_dir}/forward_data.npz ...")
data = np.load(os.path.join(input_dir, "forward_data.npz"))

# 1. Coordenadas de la malla deformada
coords_deformed = data['coords_deformed']

# 2. Estado físico asociado
Gamma_obs_data = data['Gamma_data']  
P_alv_final = float(data['P_alv_final'])

print(f"Entradas cargadas:")
print(f"  - Geometría: {coords_deformed.shape[0]} nodos")
print(f"  - Carga externa: P_alv = {P_alv_final*1000:.1f} Pa")

# ============================================================
# 3. CONSTRUCCIÓN DE LA MALLA OBSERVADA
# ============================================================

Lx, Ly = 10.0, 10.0
nx, ny = 20, 20

# 1. Topología base
mesh_topology = fd.RectangleMesh(nx, ny, Lx, Ly)

# 2. Malla deformada (Input del problema)
mesh_obs = fd.Mesh(mesh_topology.coordinates)
mesh_obs.coordinates.dat.data[:] = coords_deformed

print("Malla deformada reconstruida exitosamente.")

# ============================================================
# 4. ESPACIOS Y VARIABLES
# ============================================================

V = fd.VectorFunctionSpace(mesh_obs, "CG", 2)
Q = fd.FunctionSpace(mesh_obs, "DG", 0)

# Incógnita: w (Desplazamiento Inverso)
w = fd.Function(V, name="Inverse_Displacement")
v_test = fd.TestFunction(V)

# Campos fijos (Observados)
Gamma_field = fd.Function(Q, name="Gamma_Observed")
Gamma_field.dat.data[:] = Gamma_obs_data
alpha_load = fd.Constant(0.0) 

# ============================================================
# 5. FORMULACIÓN (CINEMÁTICA Y EQUILIBRIO)
# ============================================================

dim = mesh_obs.geometric_dimension()
I = fd.Identity(dim)

# Cinemática Inversa: X = x - w
f_inv = I - fd.grad(w)
det_f = fd.det(f_inv)
F = fd.inv(f_inv) 
J = fd.det(F)

# Constitutivo
C = F.T * F
invC = fd.inv(C)
E = fd.variable(0.5 * (C - I))

J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))
Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Cargas Externas
ratio = Gamma_field / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma_base = fd.conditional(fd.lt(Gamma_field, Gamma_inf), gamma_lang, gamma_insol)
gamma_base = fd.max_value(gamma_base, gamma_min)

gamma_eff = alpha_load * gamma_base
Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)
P_gamma = (2.0 * gamma_eff / R_RI) * ((Phi_R / Phi)**(1.0/3.0))
P_alv_eff = alpha_load * P_alv_final

S_ext = (P_gamma - P_alv_eff) * J * invC
S_total = S_el + S_ext

# Residuo
P_stress = F * S_total
grad_v_ref = F.T * fd.grad(v_test)
Residual = fd.inner(P_stress, grad_v_ref) * det_f * fd.dx

bcs = [
    fd.DirichletBC(V, fd.Constant((0.0, 0.0)), 1),
    fd.DirichletBC(V.sub(1), 0.0, 3)
]

# ============================================================
# 6. RESOLUCIÓN
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, w, bcs=bcs)
solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt", 
    "snes_max_it": 50,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
    "snes_monitor": None
}
solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

w.assign(fd.Constant((0.0, 0.0)))
print("\nIniciando cálculo inverso...")
n_steps = 10
success = True

for step in range(n_steps + 1):
    alpha_val = step / n_steps
    alpha_load.assign(alpha_val)
    print(f"--- Carga {step}/{n_steps}: {alpha_val*100:.0f}% ---")
    try:
        solver.solve()
        w_max = np.max(np.abs(w.dat.data_ro))
        print(f"    Convergió. w_max: {w_max:.4f} mm")
    except fd.ConvergenceError:
        print(f"!!! Falló convergencia !!!")
        success = False
        break

# ============================================================
# 7. GENERACIÓN DE RESULTADOS PARA PARAVIEW
# ============================================================

output_dir = "phase2_results"
os.makedirs(output_dir, exist_ok=True)

if success:
    print("\n" + "="*60)
    print("GUARDANDO RESULTADO FINAL")
    print("="*60)
    
    # --- 1. Crear el vector de RECUPERACIÓN ---
    # Para "Warp by Vector" en Paraview, necesitamos un vector tal que:
    # Pos_Final = Pos_Inicial + Vector
    # Queremos: Pos_Final = StressFree
    # Sabemos: StressFree = Deformada - w
    # Por tanto: StressFree = Deformada + (-w)
    # EL VECTOR ES -w
    
    u_recovery = fd.Function(V, name="Recovery_Vector")
    u_recovery.assign(-1.0 * w)
    
    # --- 2. Guardar en formato PVD sobre la malla deformada ---
    outfile = VTKFile(os.path.join(output_dir, "inverse_visual_ready.pvd"))
    outfile.write(u_recovery, w)
    
    print(f"Archivo generado: {output_dir}/inverse_visual_ready.pvd")
    print("\n*** INSTRUCCIONES PARA PARAVIEW ***")
    print("1. Abre 'inverse_visual_ready.pvd'.")
    print("2. Verás la malla DEFORMADA (expandida).")
    print("3. Aplica el filtro 'Warp By Vector'.")
    print("4. Selecciona 'Vectors': 'Recovery_Vector'.")
    print("5. Deja 'Scale Factor': 1.0.")
    print("6. Dale a 'Apply' -> La malla volverá a ser un cuadrado.")
    
    # --- 3. Verificación Numérica (Solución al error de broadcasting) ---
    # Interpolamos w a CG1 (lineal) para poder restarlo de las coordenadas (CG1)
    V_coords = mesh_obs.coordinates.function_space()
    w_cg1 = fd.Function(V_coords)
    w_cg1.interpolate(w)
    
    # Calculamos coordenadas recuperadas
    coords_rec = mesh_obs.coordinates.dat.data_ro - w_cg1.dat.data_ro
    
    # Construimos malla temporal para medir área
    mesh_rec_check = fd.Mesh(mesh_topology.coordinates)
    mesh_rec_check.coordinates.dat.data[:] = coords_rec
    
    vol_obs = fd.assemble(fd.Constant(1.0)*fd.dx(domain=mesh_obs))
    vol_rec = fd.assemble(fd.Constant(1.0)*fd.dx(domain=mesh_rec_check))
    
    print(f"\nVerificación:")
    print(f"  Área Inicial (Deformada): {vol_obs:.2f} mm²")
    print(f"  Área Recuperada (Calculada): {vol_rec:.2f} mm²")
    
else:
    print("\nEl cálculo falló.")

FASE 2: INVERSE MOTION SOBRE MALLA DEFORMADA
Cargando datos desde phase1_results/forward_data.npz ...
Entradas cargadas:
  - Geometría: 441 nodos
  - Carga externa: P_alv = 1000.0 Pa
Malla deformada reconstruida exitosamente.

Iniciando cálculo inverso...
--- Carga 0/10: 0% ---
  0 SNES Function norm 0.000000000000e+00
    Convergió. w_max: 0.0000 mm
--- Carga 1/10: 10% ---
  0 SNES Function norm 3.075869615248e-01
  1 SNES Function norm 1.142434338038e-01
  2 SNES Function norm 1.063075962059e-01
  3 SNES Function norm 9.108184223706e-02
  4 SNES Function norm 4.620321347113e-02
  5 SNES Function norm 3.312500506346e-02
  6 SNES Function norm 4.928043268192e-03
  7 SNES Function norm 1.638995647707e-04
  8 SNES Function norm 6.934840453414e-08
  9 SNES Function norm 1.262946092093e-14
    Convergió. w_max: 0.4914 mm
--- Carga 2/10: 20% ---
  0 SNES Function norm 3.082200787596e-01
  1 SNES Function norm 6.959737912085e-02
  2 SNES Function norm 4.051320957673e-02
  3 SNES Function nor

In [ ]:
"""
FASE 3: Aplicación a Pulmón Real desde CT
==========================================

Este script muestra cómo aplicar la metodología de recuperación
de configuración stress-free a una malla de pulmón real.

Flujo de trabajo:
1. Cargar malla del pulmón (desde CT)
2. Definir condiciones de borde anatómicas
3. Estimar parámetros de presión pleural
4. Resolver problema inverso
5. Obtener configuración stress-free

Nota: La malla del pulmón debe estar en formato compatible con Firedrake
(por ejemplo, Gmsh .msh)
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
from firedrake.adjoint import *
import numpy as np
from firedrake.output import VTKFile

# ============================================================
# 1. PARÁMETROS DEL MODELO
# ============================================================

# Geometría alveolar (Tabla 1 del paper)
Phi_R = 0.74          # Porosidad de referencia [-]
R_RI = 0.11           # Radio interno de referencia [mm]

# Material Fung
c_fung = 2.5          # [kPa] - valor original del paper
a_fung = 0.433        # [-]
b_fung = -0.61        # [-]

# Surfactante
gamma_0 = 70.0e-6     # [N/mm]
gamma_inf = 22.2e-6   # [N/mm]
gamma_min = 0.0       # [N/mm]
Gamma_inf = 3.0e-9    # [g/mm²]
m1 = 47.8e-6          # [N/mm]
m2 = 140.0e-6         # [N/mm]
Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

# Presión pleural típica en reposo
P_pleural = -0.5      # [kPa] ~ -5 cm H2O (negativa = succión)

print("="*60)
print("FASE 3: RECUPERACIÓN STRESS-FREE EN PULMÓN REAL")
print("="*60)

# ============================================================
# 2. CARGAR MALLA DEL PULMÓN
# ============================================================
"""
En la práctica, la malla vendría de:
1. Segmentación de CT
2. Generación de malla con CGAL, TetGen, o Gmsh
3. Conversión a formato compatible

Aquí usamos una malla simplificada como ejemplo.
Para malla real, usar:
    mesh = fd.Mesh("lung_mesh.msh")
"""

def create_example_lung_mesh():
    """
    Crear malla simplificada tipo pulmón (elipsoide)
    En práctica, cargar desde archivo.
    """
    # Dimensiones aproximadas de un pulmón
    Lx, Ly, Lz = 120.0, 150.0, 200.0  # mm
    nx, ny, nz = 10, 12, 16
    
    # Malla rectangular (simplificación)
    mesh = fd.BoxMesh(nx, ny, nz, Lx, Ly, Lz)
    
    return mesh, Lx, Ly, Lz

# Cargar o crear malla
print("\nCargando malla del pulmón...")

# Opción 1: Malla de ejemplo
mesh, Lx, Ly, Lz = create_example_lung_mesh()

# Opción 2: Cargar malla real (descomentar si disponible)
# mesh = fd.Mesh("lung_mesh.msh")

n_cells = mesh.num_cells()
n_vertices = mesh.num_vertices()
print(f"  Malla cargada: {n_cells} elementos, {n_vertices} vértices")

# ============================================================
# 3. ESPACIOS DE FUNCIONES
# ============================================================

dim = mesh.geometric_dimension()
I = fd.Identity(dim)
x = fd.SpatialCoordinate(mesh)

# Espacios
V = fd.VectorFunctionSpace(mesh, "CG", 1)  # Grado 1 para eficiencia en 3D
Q = fd.FunctionSpace(mesh, "DG", 0)

print(f"  Espacio V: {V.dim()} DoFs")
print(f"  Espacio Q: {Q.dim()} DoFs")

# ============================================================
# 4. CONDICIONES DE BORDE ANATÓMICAS
# ============================================================
"""
Bordes del pulmón:
1. Vías aéreas (bronquios principales) - presión de aire
2. Pleura visceral - presión pleural
3. Hilio pulmonar - fijo (conexión con mediastino)

Para el problema inverso:
- Los puntos cerca del hilio están fijos (w = 0)
- La pleura está sujeta a presión pleural
"""

def define_anatomical_regions(mesh, Lx, Ly, Lz):
    """
    Definir regiones anatómicas para condiciones de borde
    
    En malla real, esto vendría de etiquetas del mallador
    """
    x = fd.SpatialCoordinate(mesh)
    
    # Región del hilio (cerca de y = Ly, z ≈ Lz/2)
    # En malla real: marcador específico
    hilum_region = fd.conditional(
        fd.And(fd.gt(x[1], 0.9*Ly), 
               fd.And(fd.gt(x[2], 0.4*Lz), fd.lt(x[2], 0.6*Lz))),
        1.0, 0.0
    )
    
    return hilum_region

hilum_marker = define_anatomical_regions(mesh, Lx, Ly, Lz)

# Condiciones de borde
# Para malla real con marcadores:
#   bcs = [fd.DirichletBC(V, Constant((0,0,0)), "hilum")]

# Para malla de ejemplo, fijamos una cara
bcs = [
    fd.DirichletBC(V, fd.Constant((0.0, 0.0, 0.0)), 2),  # y = Ly (hilio)
]

# ============================================================
# 5. VARIABLES DEL PROBLEMA INVERSO
# ============================================================

# Desplazamiento inverso (incógnita)
w = fd.Function(V, name="inverse_displacement")
v_test = fd.TestFunction(V)

# Concentración de surfactante (asumida uniforme inicialmente)
Gamma = fd.Function(Q, name="surfactant")
Gamma.assign(fd.Constant(0.5 * (Gamma_inf + Gamma_max)))  # Estado intermedio

# Presión
P_alv = fd.Constant(-P_pleural)  # Presión transmural

print(f"\nCondiciones del problema:")
print(f"  P_pleural = {P_pleural} kPa")
print(f"  Γ inicial = {float(Gamma.dat.data_ro.mean()):.2e} g/mm²")

# ============================================================
# 6. MODELO CONSTITUTIVO 3D
# ============================================================

def inverse_kinematics_3d(w_field):
    """Cinemática inversa en 3D"""
    grad_w = fd.grad(w_field)
    f = I - grad_w
    det_f = fd.det(f)
    F = fd.inv(f)
    return F, f, det_f

def constitutive_model_3d(F, Gamma_field, P_alv_const):
    """Modelo constitutivo 3D con surfactante"""
    J = fd.det(F)
    C = F.T * F
    invC = fd.inv(C)
    
    E = fd.variable(0.5 * (C - I))
    
    # Invariantes 3D
    J1 = fd.tr(E)
    J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))
    
    # Fung
    Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
    S_el = fd.diff(Psi_el, E)
    
    # Porosidad
    Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)
    
    # Tensión superficial
    ratio = Gamma_field / Gamma_inf
    gamma_lang = gamma_0 - m1 * ratio
    gamma_insol = gamma_inf - m2 * (ratio - 1.0)
    gamma = fd.conditional(fd.lt(Gamma_field, Gamma_inf), gamma_lang, gamma_insol)
    gamma = fd.max_value(gamma, gamma_min)
    
    # Presión de colapso
    P_gamma = (2.0 * gamma / R_RI) * ((Phi_R / Phi)**(1.0/3.0))
    
    # Tensor total
    S = S_el + (P_gamma - P_alv_const) * J * invC
    
    return S, J, gamma, P_gamma

# ============================================================
# 7. FORMULACIÓN VARIACIONAL
# ============================================================

def equilibrium_residual_3d(w_field, Gamma_field, P_alv_const, test_func):
    """Residuo de equilibrio en 3D"""
    F, f, det_f = inverse_kinematics_3d(w_field)
    S, J, gamma, P_gamma = constitutive_model_3d(F, Gamma_field, P_alv_const)
    
    P = F * S
    grad_v_ref = F.T * fd.grad(test_func)
    det_f_safe = fd.max_value(det_f, 1e-6)
    
    Residual = fd.inner(P, grad_v_ref) * det_f_safe * fd.dx
    
    return Residual

Residual = equilibrium_residual_3d(w, Gamma, P_alv, v_test)

# ============================================================
# 8. SOLVER
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, w, bcs=bcs)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",  # Backtracking para 3D
    "snes_monitor": None,
    "snes_max_it": 50,
    "snes_atol": 1e-6,
    "snes_rtol": 1e-5,
    "snes_stol": 1e-8,
    "ksp_type": "gmres",
    "ksp_max_it": 200,
    "pc_type": "ilu",  # ILU para problemas grandes
    # Para problemas muy grandes, usar:
    # "pc_type": "hypre",
    # "pc_hypre_type": "boomeramg",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# ============================================================
# 9. RESOLVER CON CONTINUACIÓN DE CARGA
# ============================================================

print("\n" + "="*60)
print("RESOLVIENDO PROBLEMA INVERSO 3D")
print("="*60)

# Inicializar
w.assign(fd.Constant((0.0, 0.0, 0.0)))

# Continuación de carga para mejor convergencia
n_steps = 5
P_final = float(P_alv)

print(f"\nContinuación de carga en {n_steps} pasos...")

output_dir = "phase3_results"
os.makedirs(output_dir, exist_ok=True)

success = True
for step in range(1, n_steps + 1):
    # Incrementar presión gradualmente
    P_current = P_final * step / n_steps
    P_alv.assign(P_current)
    
    print(f"\n  Paso {step}/{n_steps}: P = {P_current:.4f} kPa")
    
    try:
        solver.solve()
        
        # Estadísticas
        w_norm = fd.norm(w)
        F_field, _, det_f_field = inverse_kinematics_3d(w)
        J_val = fd.assemble(fd.det(F_field) * fd.dx) / fd.assemble(fd.Constant(1.0) * fd.dx)
        
        print(f"    ✓ Convergió: ||w|| = {w_norm:.4e}, J_avg = {J_val:.4f}")
        
    except fd.ConvergenceError:
        print(f"    ✗ No convergió")
        success = False
        break

# ============================================================
# 10. POST-PROCESAMIENTO
# ============================================================

if success:
    print("\n" + "="*60)
    print("ANÁLISIS DE RESULTADOS")
    print("="*60)
    
    # Campos derivados
    F_field, f_field, det_f_field = inverse_kinematics_3d(w)
    
    J_field = fd.Function(Q, name="Jacobian")
    J_field.assign(fd.project(fd.det(F_field), Q))
    
    det_f_proj = fd.Function(Q, name="det_f")
    det_f_proj.assign(fd.project(det_f_field, Q))
    
    print(f"\nEstadísticas del mapeo:")
    print(f"  ||w|| = {fd.norm(w):.4e} mm")
    print(f"  J (forward): [{J_field.dat.data_ro.min():.4f}, {J_field.dat.data_ro.max():.4f}]")
    print(f"  det(f): [{det_f_proj.dat.data_ro.min():.4f}, {det_f_proj.dat.data_ro.max():.4f}]")
    
    # Cambio de volumen total
    V_ref = fd.assemble(det_f_field * fd.dx)  # Volumen de referencia
    V_obs = fd.assemble(fd.Constant(1.0) * fd.dx)  # Volumen observado
    
    print(f"\nVolúmenes:")
    print(f"  V_observado = {V_obs:.2f} mm³ = {V_obs/1e6:.4f} L")
    print(f"  V_stress-free = {V_ref:.2f} mm³ = {V_ref/1e6:.4f} L")
    print(f"  Ratio: {V_obs/V_ref:.4f}")
    
    # Verificación stress-free
    print("\n--- Verificación stress-free (P = 0) ---")
    P_zero = fd.Constant(0.0)
    S_check, _, _, _ = constitutive_model_3d(F_field, Gamma, P_zero)
    S_norm = fd.project(fd.sqrt(fd.inner(S_check, S_check)), Q)
    
    print(f"  ||S|| con P=0: [{S_norm.dat.data_ro.min():.4e}, {S_norm.dat.data_ro.max():.4e}]")
    
    # ============================================================
    # 11. CREAR MALLA STRESS-FREE
    # ============================================================
    """
    Para crear la malla de la configuración stress-free:
    1. X = x - w(x) para cada nodo
    2. Actualizar coordenadas de la malla
    """
    
    print("\n--- Creando malla stress-free ---")
    
    # Obtener coordenadas de los nodos
    coords_obs = mesh.coordinates.dat.data_ro.copy()
    
    # Interpolar w en los nodos de la malla
    V_coords = fd.VectorFunctionSpace(mesh, "CG", 1)
    w_interp = fd.interpolate(w, V_coords)
    w_at_nodes = w_interp.dat.data_ro
    
    # Calcular coordenadas stress-free
    coords_stressfree = coords_obs - w_at_nodes
    
    print(f"  Coordenadas observadas: x ∈ [{coords_obs.min():.2f}, {coords_obs.max():.2f}]")
    print(f"  Coordenadas stress-free: X ∈ [{coords_stressfree.min():.2f}, {coords_stressfree.max():.2f}]")
    
    # Crear nueva malla con coordenadas transformadas
    # (En Firedrake, modificar coordenadas directamente en la malla)
    mesh_stressfree = fd.Mesh(mesh.coordinates.copy())
    mesh_stressfree.coordinates.dat.data[:] = coords_stressfree
    
    # ============================================================
    # 12. GUARDAR RESULTADOS
    # ============================================================
    
    # Guardar malla observada con campos
    outfile_obs = VTKFile(os.path.join(output_dir, "lung_observed.pvd"))
    w.rename("inverse_displacement")
    outfile_obs.write(w, J_field, S_norm)
    
    # Guardar malla stress-free
    V_sf = fd.VectorFunctionSpace(mesh_stressfree, "CG", 1)
    w_sf = fd.Function(V_sf, name="zero_displacement")  # w=0 en config stress-free
    
    outfile_sf = VTKFile(os.path.join(output_dir, "lung_stressfree.pvd"))
    outfile_sf.write(w_sf)
    
    # Datos numéricos
    np.savez(os.path.join(output_dir, "lung_inverse_data.npz"),
             w_data=w.dat.data_ro[:],
             coords_obs=coords_obs,
             coords_stressfree=coords_stressfree,
             J_data=J_field.dat.data_ro[:],
             V_obs=V_obs,
             V_ref=V_ref)
    
    print(f"\nResultados guardados en: {output_dir}/")
    print("  - lung_observed.pvd: malla observada con campos")
    print("  - lung_stressfree.pvd: malla stress-free")
    print("  - lung_inverse_data.npz: datos numéricos")

else:
    print("\n⚠ El solver no convergió.")
    print("Sugerencias:")
    print("  1. Reducir la presión inicial")
    print("  2. Aumentar número de pasos de continuación")
    print("  3. Refinar la malla")
    print("  4. Ajustar parámetros del material")

# ============================================================
# 13. RESUMEN Y SIGUIENTE PASOS
# ============================================================

print("\n" + "="*60)
print("FASE 3 COMPLETADA")
print("="*60)

print("""
RESUMEN DEL FLUJO DE TRABAJO:
============================

1. FASE 1: Problema directo en lámina 2D
   - Validar modelo con surfactante
   - Generar datos sintéticos
   
2. FASE 2: Problema inverso en lámina 2D  
   - Recuperar configuración stress-free
   - Validar metodología de movimiento inverso
   
3. FASE 3: Aplicación a pulmón 3D
   - Cargar malla de CT
   - Resolver problema inverso
   - Obtener geometría stress-free

APLICACIONES:
=============
- Simulación de ventilación mecánica desde config. real
- Análisis de esfuerzos pulmonares
- Planificación de tratamientos
- Modelado personalizado

LIMITACIONES:
=============
- Asume material homogéneo
- No incluye árbol bronquial explícito
- Surfactante simplificado (sin cinética Langmuir completa)
- Requiere estimación de presión pleural
""")
